## 1. Setup & Connection
### Imports  
Load all Python libraries used in Task 1:  
* **pandas / geopandas** for data wrangling  
* **sqlalchemy + psycopg2** to talk to Postgres/PostGIS  
* **geoalchemy2.Geometry** helper so geopandas can write geometries  
* a few plotting + shapely utilities for quick inspection


In [26]:
import pandas as pd 
import geopandas as gpd 
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt 

### Build a reusable PostgreSQL connector  
`pgconnect()` opens an SQLAlchemy engine from *Credentials.json*.  
The helper `query()` lets us run quick SQL and return a dataframe.  



In [28]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

### Connect to the database  
Runs `pgconnect()` – if you see **“Connected successfully.”** the engine + connection are ready.


In [30]:
db, conn = pgconnect(credentials)

Connected successfully.


### Sanity check: confirm PostGIS is installed  
Queries `PostGIS_Version()`; if it returns a version string, spatial functions are available.


In [32]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.4 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


## 2. Load Non-Spatial Tables
###  Read Business, Population, Income CSVs 
Read the three non-spatial CSV files into pandas dataframes.  
At this point we only preview them; they get written to Postgres later with `to_sql()`.


In [34]:
business = pd.read_csv("data/Businesses.csv")
business.head()

,industry_code,industry_name,sa2_code,sa2_name,0_to_50k_businesses,50k_to_200k_businesses,200k_to_2m_businesses,2m_to_5m_businesses,5m_to_10m_businesses,10m_or_more_businesses,total_businesses
0,A,"Agriculture, Forestry and Fishing",101021007,Braidwood,136,92,63,4,0,0,296
1,A,"Agriculture, Forestry and Fishing",101021008,Karabar,6,3,0,0,0,0,9
2,A,"Agriculture, Forestry and Fishing",101021009,Queanbeyan,6,4,3,0,0,3,15
3,A,"Agriculture, Forestry and Fishing",101021010,Queanbeyan - East,0,3,0,0,0,0,3
4,A,"Agriculture, Forestry and Fishing",101021012,Queanbeyan West - Jerrabomberra,7,4,5,0,0,0,16


In [35]:
income = pd.read_csv("data/income.csv")
income.head()

,sa2_code21,sa2_name,earners,median_age,median_income,mean_income
0,101021007,Braidwood,2467,51,46640,68904
1,101021008,Karabar,5103,42,65564,69672
2,101021009,Queanbeyan,7028,39,63528,69174
3,101021010,Queanbeyan - East,3398,39,66148,74162
4,101021012,Queanbeyan West - Jerrabomberra,8422,44,78630,91981


In [36]:
population = pd.read_csv("data/Population.csv")
population.head()

,sa2_code,sa2_name,0-4_people,5-9_people,10-14_people,15-19_people,20-24_people,25-29_people,30-34_people,35-39_people,...,45-49_people,50-54_people,55-59_people,60-64_people,65-69_people,70-74_people,75-79_people,80-84_people,85-and-over_people,total_people
0,102011028,Avoca Beach - Copacabana,424,522,623,552,386,222,306,416,...,572,602,570,520,464,369,226,142,70,7530
1,102011029,Box Head - MacMasters Beach,511,666,702,592,461,347,420,535,...,749,749,794,895,863,925,603,331,264,11052
2,102011030,Calga - Kulnura,200,225,258,278,274,227,214,286,...,325,436,422,397,327,264,190,100,75,4748
3,102011031,Erina - Green Point,683,804,880,838,661,502,587,757,...,859,882,901,930,917,1065,976,773,1028,14803
4,102011032,Gosford - Springfield,1164,1044,1084,1072,1499,1864,1750,1520,...,1330,1241,1377,1285,1166,949,664,476,537,21346


### Define Table Schemas in SQL  
* Raw SQL (**cell 7**) creates three empty spatial tables  
  (`sa2_regions`, `stops`, `schools`) with geometry columns, SRID 4326,  



In [38]:
from sqlalchemy import text 
create_sa2_sql = """
DROP TABLE IF EXISTS public.sa2_regions;
CREATE TABLE public.sa2_regions (
    sa2_code_2021 TEXT PRIMARY KEY,
    sa2_name_2021 TEXT,
    geometry      GEOMETRY(MULTIPOLYGON,4326)
);
CREATE INDEX IF NOT EXISTS sa2_regions_geom_idx
    ON public.sa2_regions USING GIST (geometry);

DROP TABLE IF EXISTS public.stops;
CREATE TABLE public.stops (
    stop_id   TEXT PRIMARY KEY,
    stop_name TEXT,
    geometry  GEOMETRY(POINT,4326)
);
CREATE INDEX IF NOT EXISTS stops_geom_idx
    ON public.stops USING GIST (geometry);

"""

conn.execute(text(create_sa2_sql))


### Load CSVs into Postgres with Pandas
  primary keys, and GIST indexes.  
* **cell 8** loops through *Businesses / Population / Income* dataframes  
  and writes them to Postgres with `to_sql()`, then adds b-tree indexes on `sa2_code`.

In [40]:
for csv_file, table in [
    ("data/Businesses.csv", "businesses"),
    ("data/Population.csv", "population"),
    ("data/Income.csv", "income"),
]:
    df = pd.read_csv(csv_file)
    df.drop_duplicates(inplace=True)
    df.to_sql(
        table,
        conn,
        schema = "public",
        if_exists = "replace",
        index = False,
        method = "multi")
    print(f"Loaded {table}: {len(df)} rows")


Loaded businesses: 12217 rows
Loaded population: 373 rows
Loaded income: 642 rows


In [41]:
# Method A: using your helper
cols = query(conn, """
    SELECT column_name
    FROM information_schema.columns
    WHERE table_name='income'
      AND table_schema='public';
""")
print(cols)

# Method B: manually fetch
result = conn.execute(text("""
    SELECT column_name
    FROM information_schema.columns
    WHERE table_name='income'
      AND table_schema='public';
"""))
print([row[0] for row in result.fetchall()])

     column_name
0     sa2_code21
1       sa2_name
2        earners
3     median_age
4  median_income
5    mean_income
['sa2_code21', 'sa2_name', 'earners', 'median_age', 'median_income', 'mean_income']


### Add b-tree indexes on `sa2_code`  
Speeds up joins between attribute tables and spatial layers.


In [43]:
from sqlalchemy import text

conn.execute(text("""
    CREATE INDEX IF NOT EXISTS businesses_sa2_idx
      ON public.businesses (sa2_code);
    CREATE INDEX IF NOT EXISTS population_sa2_idx
      ON public.population (sa2_code);
    CREATE INDEX IF NOT EXISTS income_sa2_idx
      ON public.income (sa2_code21);
"""))
print("Indexes created on sa2_code for all three tables.")



Indexes created on sa2_code for all three tables.


## 3. Load Spatial Tables
### Load SA2 boundaries shapefile  
* Reads `SA2_2021_AUST_GDA2020.shp` with `gpd.read_file()`.  
* Keeps only code, name, geometry; reprojects to WGS-84 (EPSG 4326).


In [45]:
# --- Load SA2 boundaries WITH SA4 + GCC columns --------------------------
import geopandas as gpd
from geoalchemy2 import Geometry
from sqlalchemy import text

# 1. Read shapefile and keep the extra admin columns
sa2 = (
    gpd.read_file("data/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")[
        ["SA2_CODE21", "SA2_NAME21", "SA4_NAME21", "GCC_NAME21", "geometry"]
    ]
    .rename(
        columns={
            "SA2_CODE21": "sa2_code",
            "SA2_NAME21": "sa2_name",
            "SA4_NAME21": "sa4_name_2021",
            "GCC_NAME21": "gcc_name_2021",
        }
    )
    .to_crs(4326)
    .drop_duplicates()
)

# 2. Re-create the table from scratch
conn.execute(text("DROP TABLE IF EXISTS public.sa2_regions;"))

sa2.to_postgis(
    "sa2_regions", conn,
    schema="public",
    if_exists="replace",            # overwrite
    index=False,
    dtype={"geometry": Geometry("MULTIPOLYGON", 4326)}
)

# 3. Add PK and spatial index
conn.execute(text("""
ALTER TABLE public.sa2_regions
  ADD CONSTRAINT sa2_regions_pkey PRIMARY KEY (sa2_code);
CREATE INDEX sa2_regions_geom_idx
  ON public.sa2_regions USING GIST (geometry);
"""))

print(f"Reloaded {len(sa2):,} SA2 polygons with SA4 & GCC info.")


Reloaded 2,473 SA2 polygons with SA4 & GCC info.


### Create Spatial Table for SA2 
Drops any old copy and recreates `public.sa2_regions` with  
`GEOMETRY(MULTIPOLYGON,4326)`, primary key `sa2_code_2021`, and GIST index.


In [47]:
ddl = """
DROP TABLE IF EXISTS public.sa2_regions;
CREATE TABLE public.sa2_regions (
    sa2_code TEXT PRIMARY KEY,
    sa2_name TEXT,
    geometry GEOMETRY(MULTIPOLYGON,4326)
);
CREATE INDEX IF NOT EXISTS sa2_regions_geom_idx
  ON public.sa2_regions USING GIST (geometry);
"""
conn.execute(text(ddl))
print("Spatial table sa2_regions created.")

Spatial table sa2_regions created.


### Insert SA2 to PostGIS 
`to_postgis()` streams the GeoDataFrame into the empty table.


In [49]:
sa2.to_postgis(
    "sa2_regions", conn,
    schema="public",
    if_exists="replace",   
    index=False,
    dtype={"geometry": Geometry("MULTIPOLYGON", 4326)}
)
print(f"Inserted {len(sa2):,} SA2 rows.")

Inserted 2,473 SA2 rows.


### Ensure SA2 Table Index Exists
PL/pgSQL block only adds them if missing (useful if the cell is rerun).


In [51]:
from sqlalchemy import text

conn.execute(text("""
DO $$
BEGIN
  -- add PK only if it doesn't exist
  IF NOT EXISTS (
      SELECT 1 FROM pg_constraint
      WHERE conname = 'sa2_regions_pkey'
  ) THEN
      ALTER TABLE public.sa2_regions
        ADD CONSTRAINT sa2_regions_pkey PRIMARY KEY (sa2_code);
  END IF;

  -- add GIST index only if it doesn't exist
  IF NOT EXISTS (
      SELECT 1 FROM pg_indexes
      WHERE schemaname = 'public'
        AND tablename  = 'sa2_regions'
        AND indexname  = 'sa2_regions_geom_idx'
  ) THEN
      CREATE INDEX sa2_regions_geom_idx
        ON public.sa2_regions USING GIST (geometry);
  END IF;
END $$;
"""))

print("Primary key and GIST index verified/created.")



Primary key and GIST index verified/created.


### Select SA2 polygons for my SA4 zone  
The ABS shapefile stores the SA4 each SA2 belongs to.  
Here we keep only SA2s that are:
* inside the **Greater Sydney** GCC, and  
* inside the SA4 zone we choose (set in `TARGET_SA4`).  

The resulting GeoDataFrame `my_sa2` will drive every later step
(spatial joins, POI API calls, scoring …).


In [53]:
pd.read_sql("""
SELECT DISTINCT sa4_name_2021
FROM   public.sa2_regions
WHERE  gcc_name_2021 = 'Greater Sydney'
ORDER  BY sa4_name_2021;
""", conn)


,sa4_name_2021
0,Central Coast
1,Sydney - Baulkham Hills and Hawkesbury
2,Sydney - Blacktown
3,Sydney - City and Inner South
4,Sydney - Eastern Suburbs
5,Sydney - Inner South West
6,Sydney - Inner West
7,Sydney - North Sydney and Hornsby
8,Sydney - Northern Beaches
9,Sydney - Outer South West


## Team Memember Hardik Chojar Chose "Sydney - Parramatta"

In [55]:
TARGET_SA4 = "Sydney - Parramatta"     

my_sa2 = gpd.read_postgis(
    """
    SELECT sa2_code,
           sa2_name,
           geometry
    FROM   public.sa2_regions
    WHERE  gcc_name_2021 = 'Greater Sydney'
      AND  sa4_name_2021 = %(sa4)s
    """,
    conn,
    geom_col="geometry",
    params={"sa4": TARGET_SA4}
)

print(f"{len(my_sa2)} SA2s in {TARGET_SA4}")


34 SA2s in Sydney - Parramatta


## 4. Load Public Transport Stops
### Load GTFS Stops and Convert to Geometry  
* Reads `Stops.txt`, constructs a GeoDataFrame from lon/lat via  
  `gpd.points_from_xy()`.  
* Appends rows to `public.stops`.


In [57]:
import pandas as pd, geopandas as gpd
from geoalchemy2 import Geometry

# build GeoDataFrame
stops = pd.read_csv("data/Stops.txt")
stops.drop_duplicates(inplace=True)

stops_gdf = gpd.GeoDataFrame(
    stops[["stop_id","stop_name"]],
    geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
    crs=4326
)

# append rows
stops_gdf.to_postgis("stops", conn,
                     schema="public",
                     if_exists="append",   # structure already created
                     index=False,
                     dtype={"geometry": Geometry("POINT",4326)})

print(f"Inserted {len(stops_gdf):,} stops")


Inserted 114,718 stops


## 5. Load School Catchments
### Merge Primary, Secondary, and Future Catchments
* Reads each shapefile, renames `USE_ID → school_id` and `CATCH_TYPE → stage`,  
  tags `sector='Government'`, concatenates.  
* Recreates `public.schools` with a surrogate key (`gid SERIAL`) + GIST index,  
  then inserts all polygons.


In [59]:
import geopandas as gpd
from geoalchemy2 import Geometry
import pandas as pd
from sqlalchemy import text

def read_catch(path):
    gdf = gpd.read_file(path).to_crs(4326)
    gdf = gdf.rename(columns={
        "USE_ID": "school_id",
        "CATCH_TYPE": "stage"
    })
    gdf["sector"] = "Government"
    return gdf[["school_id", "sector", "stage", "geometry"]]

# read each shapefile
gdf_primary   = read_catch("data/catchments/catchments_primary.shp")
gdf_secondary = read_catch("data/catchments/catchments_secondary.shp")
gdf_future    = read_catch("data/catchments/catchments_future.shp")

schools = pd.concat([gdf_primary, gdf_secondary, gdf_future], ignore_index=True)

schools.drop_duplicates(inplace=True)

print(f"Total school polygons: {len(schools):,}")

# recreate spatial table with surrogate PK
conn.execute(text("""
DROP TABLE IF EXISTS public.schools;
CREATE TABLE public.schools (
    gid       SERIAL PRIMARY KEY,          -- surrogate key
    school_id TEXT,
    sector    TEXT,
    stage     TEXT,
    geometry  GEOMETRY(MULTIPOLYGON,4326)
);
CREATE INDEX schools_geom_idx
  ON public.schools USING GIST (geometry);
"""))

# insert rows
schools.to_postgis("schools", conn, schema="public",
                   if_exists="append", index=False,
                   dtype={"geometry": Geometry("MULTIPOLYGON",4326)})

print("School catchments loaded with surrogate primary key.")


Total school polygons: 2,125
School catchments loaded with surrogate primary key.


## 6. Final Validation
### Row Counts & SRID Check  
* Row counts for all six tables (ensures data really loaded).  
* SRID confirmation – every geometry table is EPSG 4326.  
Both checks pass ⇒ Task 1 satisfied.


In [61]:
import pandas as pd

pd.read_sql("""
SELECT 'sa2_regions' AS table_name, COUNT(*) AS rows FROM public.sa2_regions
UNION ALL
SELECT 'stops',        COUNT(*) FROM public.stops
UNION ALL
SELECT 'schools',      COUNT(*) FROM public.schools
UNION ALL
SELECT 'businesses',   COUNT(*) FROM public.businesses
UNION ALL
SELECT 'population',   COUNT(*) FROM public.population
UNION ALL
SELECT 'income',       COUNT(*) FROM public.income;
""", conn)




,table_name,rows
0,population,373
1,income,642
2,businesses,12217
3,sa2_regions,2473
4,schools,2125
5,stops,114718


In [62]:
pd.read_sql("""
SELECT f_table_name AS table_name, srid
FROM   geometry_columns
WHERE  f_table_schema = 'public';
""", conn)


,table_name,srid
0,stops,4326
1,sa2_regions,4326
2,schools,4326


# Task 2
## Part 1
Here are the function that returns all points of interests from the API within a specified bounding box of coordinates.

In [64]:
import requests
import pandas as pd

def get_pois_from_api(minx, miny, maxx, maxy):
    
    url = "https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer/0/query"
    
    params = {
        "where": "1=1",
        "geometry": f"{minx},{miny},{maxx},{maxy}",
        "geometryType": "esriGeometryEnvelope",
        "spatialRel": "esriSpatialRelIntersects",
        "outFields": "*",
        "returnGeometry": "true",
        "f": "json"
    }
    
    response = requests.get(url, params=params)

    if response.status_code != 200:
        print("Error fetching data:", response.status_code)
        return pd.DataFrame(), {}
    
    result = response.json()

    if "features" not in result:
        return pd.DataFrame(), result
    
    pois = []
    
    for feature in result["features"]:
        attr = feature["attributes"]
        geom = feature.get("geometry", {})
        
        pois.append({
            "POIName": attr.get("poiname", None),
            "POIType": attr.get("poitype", None),
            "POIGroup": attr.get("poigroup", None),
            "POILabel": attr.get("poilabel", None),
            "POIAltLabel": attr.get("poialtlabel", None),
            "POILabelType": attr.get("poilabeltype", None),
            "Longitude": geom.get("x", None),
            "Latitude": geom.get("y", None)
        })
    
    return pd.DataFrame(pois), result


## Part 2
loop that cycles through each SA2 region within your selected SA4 region, waits a second before executing, then runs
the function for that region’s bounding box, to find all points of interest within that SA2 region.

In [66]:
import time
from sqlalchemy import text, String, Integer, Float, DateTime

selected_sa4 = ["Sydney - Blacktown", "Sydney - Parramatta", "Sydney - Ryde"]
selected_sa4_str = ", ".join(f"'{sa4}'" for sa4 in selected_sa4)

sa2_df = pd.read_sql(f"""
SELECT sa2_code, sa2_name, sa4_name_2021, ST_Extent(geometry) AS bbox
FROM public.sa2_regions
WHERE sa4_name_2021 IN ({selected_sa4_str})
GROUP BY sa2_code, sa2_name, sa4_name_2021
""", conn)

def bbox_parse(bbox_str):
    bbox_str = bbox_str.replace("BOX(", "").replace(")", "")
    min_point, max_point = bbox_str.split(",")
    minx, miny = map(float, min_point.strip().split(" "))
    maxx, maxy = map(float, max_point.strip().split(" "))
    return minx, miny, maxx, maxy

sa2_df[["min_lon", "min_lat", "max_lon", "max_lat"]] = sa2_df["bbox"].apply(lambda x: pd.Series(bbox_parse(x)))

results = []

for index, row in sa2_df.iterrows():
    sa2_code = row["sa2_code"]
    sa2_name = row["sa2_name"]
    min_lon = row["min_lon"]
    min_lat = row["min_lat"]
    max_lon = row["max_lon"]
    max_lat = row["max_lat"]

    print(f"Fetching POIs for SA2: {sa2_name}")

    poi_df, result_p = get_pois_from_api(min_lon, min_lat, max_lon, max_lat)
    # print(result_p)

    if poi_df.empty:
        continue

    poi_df["SA2_CODE"] = sa2_code
    poi_df["SA2_NAME"] = sa2_name
    results.append(poi_df)

    time.sleep(1)

print("Finished fetching POIs.")

if results:
    final_poi_df = pd.concat(results, ignore_index=True)
    print(f"Total POIs fetched: {len(final_poi_df)}")
    display(final_poi_df.head())
    

    final_poi_df.to_sql("nsw_poi", db, if_exists="replace", index=False, dtype={
        "POIName": String(),
        "POIType": String(),
        "POIGroup": Integer(),
        "POILabel": String(),
        "POIAltLabel": String(),
        "POILabelType": String(),
        "Longitude": Float(),
        "Latitude": Float(),
        "SA2_CODE": String(),
        "SA2_NAME": String(),
        "timestamp": String()   
    })

    print("POIs successfully inserted into table 'nsw_poi'.")
else:
    print("No POIs found.")

Fetching POIs for SA2: Blacktown (East) - Kings Park
Fetching POIs for SA2: Blacktown (North) - Marayong
Fetching POIs for SA2: Doonside - Woodcroft
Fetching POIs for SA2: Lalor Park - Kings Langley
Fetching POIs for SA2: Blacktown - South
Fetching POIs for SA2: Blacktown - West
Fetching POIs for SA2: Seven Hills - Prospect
Fetching POIs for SA2: Toongabbie - West
Fetching POIs for SA2: Glenwood
Fetching POIs for SA2: Acacia Gardens
Fetching POIs for SA2: Quakers Hill
Fetching POIs for SA2: Kellyville Ridge - The Ponds
Fetching POIs for SA2: Marsden Park - Shanes Park
Fetching POIs for SA2: Riverstone
Fetching POIs for SA2: Schofields (West) - Colebee
Fetching POIs for SA2: Schofields - East
Fetching POIs for SA2: Stanhope Gardens - Parklea
Fetching POIs for SA2: Bidwill - Hebersham - Emerton
Fetching POIs for SA2: Glendenning - Dean Park
Fetching POIs for SA2: Hassall Grove - Plumpton
Fetching POIs for SA2: Lethbridge Park - Tregear
Fetching POIs for SA2: Mount Druitt - Whalan
Fetchin

,POIName,POIType,POIGroup,POILabel,POIAltLabel,POILabelType,Longitude,Latitude,SA2_CODE,SA2_NAME
0,BREWONGLE WALKWAY,Park,3,BREWONGLE WALKWAY,None,NAMED,150.898434,-33.777748,116011303,Blacktown (East) - Kings Park
1,None,Park,3,Park,None,GENERIC,150.902324,-33.781609,116011303,Blacktown (East) - Kings Park
2,None,Park,3,Park,None,GENERIC,150.899557,-33.781500,116011303,Blacktown (East) - Kings Park
3,None,Park,3,Park,None,GENERIC,150.899392,-33.779309,116011303,Blacktown (East) - Kings Park
4,None,Park,3,Park,None,GENERIC,150.905749,-33.781910,116011303,Blacktown (East) - Kings Park


POIs successfully inserted into table 'nsw_poi'.


# Task 3

## Businesses 
We focus on industries that directly provide everyday goods, services, and amenities to local residents.  To that end, we exclude:
	
    •	Heavy-industry & utilities (Mining, Manufacturing, Agriculture, Electricity/Gas/Water/Waste, Construction)
These sectors typically locate outside dense population centers and don’t reflect the local availability of shops, clinics, or leisure services.
	
    •	Public Administration & Safety
Although essential, these are government-run functions rather than private-sector businesses, and so are better handled by dedicated public-sector layers.
	
    •	Administrative & Support Services
Roles like cleaning, security or back-office operations don’t signal consumer-facing amenities; they rarely indicate local resource richness from a resident’s perspective.

In [69]:
from sqlalchemy import text

sql = """
DROP TABLE IF EXISTS public.sa2_summary;

CREATE TABLE public.sa2_summary AS
SELECT
  sr.sa2_code,
  sr.sa2_name,
  SUM(b.total_businesses)
    / (pop.total_people   / 1000.0) AS business_count
FROM businesses b
  JOIN sa2_regions    sr  USING (sa2_name)
  JOIN population pop USING (sa2_name)
WHERE sr.sa4_name_2021 IN (
    'Sydney - Blacktown',
    'Sydney - Parramatta',
    'Sydney - Ryde'
  )
  AND b.industry_name IN (
    'Health Care and Social Assistance',
    'Education and Training',
    'Retail Trade',
    'Accommodation and Food Services',
    'Transport, Postal and Warehousing',
    'Financial and Insurance Services',
    'Professional, Scientific and Technical Services',
    'Arts and Recreation Services',
    'Rental, Hiring and Real Estate Services'
  )
  AND total_people >= 100
GROUP BY
  sr.sa2_code,
  sr.sa2_name,
  pop.total_people
;
"""

conn.execute(text(sql))

summary_df = query(conn, "SELECT * FROM public.sa2_summary;")
print(summary_df.head())

    sa2_code                       sa2_name  business_count
0  116011303  Blacktown (East) - Kings Park       98.009545
1  116011304   Blacktown (North) - Marayong       42.186973
2  116011306           Doonside - Woodcroft       38.427319
3  116011307     Lalor Park - Kings Langley       51.711324
4  116011560              Blacktown - South       41.788220


## Stops
We capture public‐transport accessibility by counting how many transit stops lie within each SA2.  A higher stops_count indicates better local connectivity and ease of movement for residents.  

In [71]:
sql = """
-- 1) Add the new column if it doesn't already exist
ALTER TABLE public.sa2_summary
ADD COLUMN IF NOT EXISTS stops_count INTEGER;

-- 2) Update that column by counting stops per SA2
UPDATE public.sa2_summary AS summ
SET stops_count = sub.stops_count
FROM (
  SELECT
    sr.sa2_name,
    COUNT(*) AS stops_count
  FROM sa2_regions AS sr
  JOIN stops AS s
    ON ST_Contains(sr.geometry, s.geometry)
  WHERE
    sr.sa4_name_2021 IN (
      'Sydney - Blacktown',
      'Sydney - Parramatta',
      'Sydney - Ryde'
    )
  GROUP BY
    sr.sa2_name
) AS sub
WHERE summ.sa2_name = sub.sa2_name;
"""

conn.execute(text(sql))

summary_df = query(
    conn,
    "SELECT * FROM public.sa2_summary;"
)
print(summary_df.head())

    sa2_code                       sa2_name  business_count  stops_count
0  116011303  Blacktown (East) - Kings Park       98.009545          228
1  116011304   Blacktown (North) - Marayong       42.186973          234
2  116011306           Doonside - Woodcroft       38.427319          221
3  116011307     Lalor Park - Kings Langley       51.711324          345
4  116011560              Blacktown - South       41.788220           82


## Schools
To gauge local education access, we count how many distinct school‐catchment polygons actually overlap each SA2, then express that as “catchments per 1 000 young people.”  We:
	
    •	Exclude any SA2 with fewer than 100 residents aged 0–19, to avoid unstable rates.
	•	Divide the raw catchment count by (young_pop / 1 000) so that areas with very few kids don’t produce huge per-capita spikes

In [73]:
sql = """
-- 1) Add the new column if it doesn't already exist
ALTER TABLE public.sa2_summary
ADD COLUMN IF NOT EXISTS catchments_count DOUBLE PRECISION;

-- 2) Update that column by computing catchments per 1 000 young people
WITH pop_young AS (
  SELECT
    sa2_name,
    (
      "0-4_people"
    + "5-9_people"
    + "10-14_people"
    + "15-19_people"
    ) AS young_pop
  FROM population
)
UPDATE public.sa2_summary AS summ
SET catchments_count = sub.catchments_count
FROM (
  SELECT
    sr.sa2_name,
    COUNT(*)::float
      / (pyoung.young_pop / 1000.0) AS catchments_count
  FROM schools      AS s
  JOIN sa2_regions AS sr
    ON  ST_Intersects(sr.geometry, s.geometry)
    AND NOT ST_Touches(   sr.geometry, s.geometry)
  JOIN pop_young   AS pyoung
    ON pyoung.sa2_name = sr.sa2_name
  WHERE
    sr.sa4_name_2021 IN (
      'Sydney - Blacktown',
      'Sydney - Parramatta',
      'Sydney - Ryde'
    )
    AND pyoung.young_pop >= 100
  GROUP BY
    sr.sa2_name,
    pyoung.young_pop
) AS sub
WHERE summ.sa2_name = sub.sa2_name;
"""

conn.execute(text(sql))

summary_df = query(
    conn,
    "SELECT * FROM public.sa2_summary;"
)
print(summary_df.head())

    sa2_code          sa2_name  business_count  stops_count  catchments_count
0  116021562    Acacia Gardens       47.269764           39          4.708098
1  125011582  Auburn - Central       71.657428          141          2.775850
2  125011583    Auburn - North       86.384012           80          4.672897
3  125011584    Auburn - South       60.092104           83          5.577689
4  125011585            Berala       42.095524           96          4.975124


## NSW POI
To measure local amenity density, we count only those points of interest that represent community-facing services and everyday resources.  We exclude:
	
    •	Industrial or utility facilities (Filtration Plant, Sewage Works, Pumping Station) as they don’t serve residents directly.
	•	Large‐scale commercial or tourism sites (Tourist Park, Silo – Commercial, Quarry – Open Cut) which typically lie outside residential cores.
	•	Very broad or redundant places (City, Suburb, Locality) that don’t convey true amenity density.
	•	Niche or infrequent-use sites (Motor Racing Track, Helipad, Gaol) which aren’t regular community resources.

In [75]:
sql = """
-- 1) Add the new column if it doesn't already exist
ALTER TABLE public.sa2_summary
ADD COLUMN IF NOT EXISTS poi_count INTEGER;

-- 2) Update that column by counting POIs per SA2
UPDATE public.sa2_summary AS summ
SET poi_count = sub.poi_count
FROM (
  SELECT
    sr.sa2_name,
    COUNT(*) AS poi_count
  FROM nsw_poi AS n
  JOIN sa2_regions AS sr
    ON sr.sa2_name = n."SA2_NAME"
  WHERE
    sr.sa4_name_2021 IN (
      'Sydney - Blacktown',
      'Sydney - Parramatta',
      'Sydney - Ryde'
    )
    AND n."POIType" IN (
      'Library',
      'Park',
      'Community Facility',
      'Child Care Centre',
      'Primary School',
      'High School',
      'General Hospital',
      'Community Medical Centre',
      'Transport Interchange',
      'Post Office',
      'Sports Centre / Sports Field',
      'Art Gallery / Museum'
    )
  GROUP BY
    sr.sa2_name
) AS sub
WHERE summ.sa2_name = sub.sa2_name;
"""

conn.execute(text(sql))

summary_df = query(
    conn,
    "SELECT * FROM public.sa2_summary;"
)
print(summary_df.head())

    sa2_code          sa2_name  business_count  stops_count  catchments_count  \
0  116021562    Acacia Gardens       47.269764           39          4.708098   
1  125011582  Auburn - Central       71.657428          141          2.775850   
2  125011583    Auburn - North       86.384012           80          4.672897   
3  125011584    Auburn - South       60.092104           83          5.577689   
4  125011585            Berala       42.095524           96          4.975124   

   poi_count  
0          3  
1         45  
2         30  
3         23  
4         26  


## Z scores and Final Sigmoid Score
To bring all four metrics (business, stops, schools, POIs) onto a common scale and combine them into one index, we:
	
    • Standardize each metric by converting raw counts into z-scores, where mu and sigma are the mean and population standard deviation of that metric across your selected SA2s.
	
    • Sum the four z-scores into a single value, so positive values indicate above-average resource levels and negatives indicate below-average.
	
    • Apply a sigmoid transform, to squash the combined score into the (0,1) range, making it easy to interpret (closer to 1 → very well-resourced; closer to 0 → poorly-resourced).

In [77]:
sql = """
ALTER TABLE public.sa2_summary
  ADD COLUMN IF NOT EXISTS z_business          DOUBLE PRECISION,
  ADD COLUMN IF NOT EXISTS z_stops             DOUBLE PRECISION,
  ADD COLUMN IF NOT EXISTS z_schools           DOUBLE PRECISION,
  ADD COLUMN IF NOT EXISTS z_poi               DOUBLE PRECISION,
  ADD COLUMN IF NOT EXISTS z_total             DOUBLE PRECISION,
  ADD COLUMN IF NOT EXISTS well_resourced_score DOUBLE PRECISION;

WITH stats AS (
  SELECT
    AVG(business_count)      AS mean_bc,
    STDDEV_POP(business_count) AS sd_bc,
    AVG(stops_count)         AS mean_sc,
    STDDEV_POP(stops_count)    AS sd_sc,
    AVG(catchments_count)    AS mean_cc,
    STDDEV_POP(catchments_count) AS sd_cc,
    AVG(poi_count)           AS mean_pc,
    STDDEV_POP(poi_count)      AS sd_pc
  FROM public.sa2_summary
)
UPDATE public.sa2_summary
SET
  z_business = (business_count - stats.mean_bc) / NULLIF(stats.sd_bc,0),
  z_stops    = (stops_count    - stats.mean_sc) / NULLIF(stats.sd_sc,0),
  z_schools  = (catchments_count - stats.mean_cc) / NULLIF(stats.sd_cc,0),
  z_poi      = (poi_count      - stats.mean_pc) / NULLIF(stats.sd_pc,0),
  z_total    = (
      (business_count   - stats.mean_bc) / NULLIF(stats.sd_bc,0)
    + (stops_count      - stats.mean_sc) / NULLIF(stats.sd_sc,0)
    + (catchments_count - stats.mean_cc) / NULLIF(stats.sd_cc,0)
    + (poi_count        - stats.mean_pc) / NULLIF(stats.sd_pc,0)
  ),
  well_resourced_score = 1.0 / (1.0 + EXP(-(
      (business_count   - stats.mean_bc) / NULLIF(stats.sd_bc,0)
    + (stops_count      - stats.mean_sc) / NULLIF(stats.sd_sc,0)
    + (catchments_count - stats.mean_cc) / NULLIF(stats.sd_cc,0)
    + (poi_count        - stats.mean_pc) / NULLIF(stats.sd_pc,0)
  )))
FROM stats;
"""

conn.execute(text(sql))

summary_df = query(
    conn,
    """
    SELECT 
      sa2_code, 
      sa2_name, 
      z_business, 
      z_stops, 
      z_schools, 
      z_poi, 
      z_total, 
      well_resourced_score
    FROM public.sa2_summary
    """
)
print(summary_df.head())

    sa2_code          sa2_name  z_business   z_stops  z_schools     z_poi  \
0  116021562    Acacia Gardens   -0.524771 -1.606917   0.994567 -1.957806   
1  125011582  Auburn - Central    0.371759 -0.225227  -0.571959 -0.512942   
2  125011583    Auburn - North    0.913132 -1.051532   0.966029 -1.028965   
3  125011584    Auburn - South   -0.053401 -1.010894   1.699569 -1.269775   
4  125011585            Berala   -0.714985 -0.834796   1.211053 -1.166571   

    z_total  well_resourced_score  
0 -3.094926              0.043317  
1 -0.938369              0.281230  
2 -0.201335              0.449836  
3 -0.634501              0.346491  
4 -1.505299              0.181637  


# Distribution Summary for Task 4 Report
## Table of Mean, Median, Spread (Standard Deviation) of overall (obtained from task 3), and the 3 selected regions

In [79]:
# Load summary_df (already calculated scores)
summary_df = query(conn, "SELECT * FROM public.sa2_summary;")

# Load SA2 regions to get sa4_name_2021 and merge the data
sa2_regions_df = query(conn, "SELECT sa2_code, sa2_name, sa4_name_2021 FROM public.sa2_regions;")
summary_df = summary_df.merge(sa2_regions_df, on=["sa2_code", "sa2_name"], how="left")

# Calculate stats overall
overall_stats = pd.DataFrame({
    "SA4 Region": ["Overall"],
    "Mean Score": [summary_df["well_resourced_score"].mean()],
    "Median Score": [summary_df["well_resourced_score"].median()],
    "Standard Deviation": [summary_df["well_resourced_score"].std()]
})

# Calculate stats per SA4 region
cross_zone_summary = summary_df.groupby("sa4_name_2021")["well_resourced_score"].agg(["mean", "median", "std"]).reset_index()
cross_zone_summary.columns = ["SA4 Region", "Mean Score", "Median Score", "Standard Deviation"]

# Combine both Overall + Cross zone region
combined_summary = pd.concat([overall_stats, cross_zone_summary], ignore_index=True)

# Table Output
display(combined_summary)


,SA4 Region,Mean Score,Median Score,Standard Deviation
0,Overall,0.472897,0.456311,0.304489
1,Sydney - Blacktown,0.332012,0.212802,0.317255
2,Sydney - Parramatta,0.525907,0.574752,0.235973
3,Sydney - Ryde,0.595746,0.553841,0.352898


In [123]:
# Load summary_df (already calculated scores)
summary_df = query(conn, "SELECT * FROM public.sa2_summary;")

# Load SA2 regions to get sa4_name_2021 and merge the data
sa2_regions_df = query(conn, "SELECT sa2_code, sa2_name, sa4_name_2021 FROM public.sa2_regions;")
summary_df = summary_df.merge(sa2_regions_df, on=["sa2_code", "sa2_name"], how="left")

# Calculate overall stats including IQR
q1 = summary_df["well_resourced_score"].quantile(0.25)
q3 = summary_df["well_resourced_score"].quantile(0.75)
iqr = q3 - q1

overall_stats = pd.DataFrame({
    "SA4 Region": ["Overall"],
    "Mean Score": [summary_df["well_resourced_score"].mean()],
    "Median Score": [summary_df["well_resourced_score"].median()],
    "Standard Deviation": [summary_df["well_resourced_score"].std()],
    "IQR": [iqr]
})

# Calculate stats per SA4 region including IQR
def calc_iqr(group):
    q1 = group.quantile(0.25)
    q3 = group.quantile(0.75)
    return q3 - q1

cross_zone_summary = summary_df.groupby("sa4_name_2021")["well_resourced_score"].agg(
    mean="mean",
    median="median",
    std="std",
    iqr=calc_iqr
).reset_index()

cross_zone_summary.columns = ["SA4 Region", "Mean Score", "Median Score", "Standard Deviation", "IQR"]

# Combine overall and per-region stats
combined_summary = pd.concat([overall_stats, cross_zone_summary], ignore_index=True)

# Display table
display(combined_summary)


,SA4 Region,Mean Score,Median Score,Standard Deviation,IQR
0,Overall,0.472897,0.456311,0.304489,0.472755
1,Sydney - Blacktown,0.332012,0.212802,0.317255,0.415446
2,Sydney - Parramatta,0.525907,0.574752,0.235973,0.316631
3,Sydney - Ryde,0.595746,0.553841,0.352898,0.651510
